In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# 올리브영

In [4]:
# 데이터 불러오기
oliveyoung = pd.read_csv('통합_올리브영_데이터.csv', encoding="utf-8-sig")

# 데이터셋 복제
oliveyoung_df = oliveyoung.copy(deep=True)

In [5]:
# 데이터셋 기본 정보 조회
print(oliveyoung_df.info())
print(oliveyoung_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None
    플랫폼   브랜드                                                제품명  중분류_순위  \
0  올리브영  에스트라  에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...       1   
1  올리브영  에스트라            에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획       2   
2  올리브영  에스트라  에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...       3   

    대분류        중분류        소분류   평점   후기 수

## 기본 전처리

In [6]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# 원본 데이터셋에 전처리 되어있음

'''
for i in range(len(oliveyoung_df)):
  score = oliveyoung_df['평점'][i]
  review = oliveyoung_df['후기 수'][i]
  price = oliveyoung_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  oliveyoung_df.loc[i, '평점'] = score_result
  oliveyoung_df.loc[i, '후기 수'] = review_result
  oliveyoung_df.loc[i, '가격'] = price_result

print(oliveyoung_df.loc[:10, ['평점', '후기 수', '가격']])
'''

"\nfor i in range(len(oliveyoung_df)):\n  score = oliveyoung_df['평점'][i]\n  review = oliveyoung_df['후기 수'][i]\n  price = oliveyoung_df['가격'][i]\n  \n  score_result = score.replace('별점 없음', '0')\n  review_result = review.replace(',', '')\n  price_result = price.replace(',', '')\n  \n  oliveyoung_df.loc[i, '평점'] = score_result\n  oliveyoung_df.loc[i, '후기 수'] = review_result\n  oliveyoung_df.loc[i, '가격'] = price_result\n\nprint(oliveyoung_df.loc[:10, ['평점', '후기 수', '가격']])\n"

In [7]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경
# 원본 데이터셋에 전처리 되어있음 

'''
oliveyoung_df[['평점', '후기 수', '가격']] = oliveyoung_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(oliveyoung_df[['평점', '후기 수', '가격']].dtypes)
print(oliveyoung_df.loc[:5, ['평점', '후기 수', '가격']])
'''

"\noliveyoung_df[['평점', '후기 수', '가격']] = oliveyoung_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)\n\nprint(oliveyoung_df[['평점', '후기 수', '가격']].dtypes)\nprint(oliveyoung_df.loc[:5, ['평점', '후기 수', '가격']])\n"

In [8]:
# 2. 컬럼 이름 변경

oliveyoung_df.rename(columns = {'대분류_순위':'중분류_순위', '정가':'가격'}, inplace=True)
oliveyoung_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '가격',
       '할인가'],
      dtype='object')

In [9]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

'''
oliveyoung_df['플랫폼'] = '올리브영'

a = oliveyoung_df.columns[-1:].to_list()

ex_a = oliveyoung_df.drop(columns = oliveyoung_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
oliveyoung_df = oliveyoung_df[reorder]

oliveyoung_df.columns
'''

"\noliveyoung_df['플랫폼'] = '올리브영'\n\na = oliveyoung_df.columns[-1:].to_list()\n\nex_a = oliveyoung_df.drop(columns = oliveyoung_df.columns[[-1]]).columns[:].to_list()\n\nreorder = a + ex_a\noliveyoung_df = oliveyoung_df[reorder]\n\noliveyoung_df.columns\n"

## 카테고리 전처리

In [10]:
# 1. 중분류1 & 중분류2 컬럼 생성
oliveyoung_df['중분류1'] = ''
print(oliveyoung_df['중분류1'].head())

oliveyoung_df['중분류2'] = ''
print(oliveyoung_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [11]:
# 기존 분류별 unique 값 확인
print(oliveyoung_df['대분류'].unique())
print(oliveyoung_df['중분류'].unique())
print(oliveyoung_df['소분류'].unique())

['스킨케어' '클렌징' '마스크팩' '선케어' '메이크업']
['크림' '에센스/세럼/앰플' '로션' '미스트/오일' '스킨/토너' '스킨케어세트' '클렌징폼/젤' '오일/밤' '워터/밀크'
 '필링&스크럽' '립&아이리무버' '티슈/패드' '패드' '시트팩' '페이셜팩' '패치' '코팩' '선크림' '선쿠션'
 '태닝/애프터선' '선스틱' '선스프레이/선패치' '베이스메이크업' '립메이크업' '아이메이크업']
['크림' '에센스/세럼/앰플' '로션' '미스트/픽서' '스킨/토너' '올인원' '페이스오일' '스킨케어세트' '클렌징폼/젤'
 '클렌징오일' '클렌징워터' '클렌징밤' '피지클리너' '팩클렌저' '페이셜스크럽' '클렌징밀크/크림' '립&아이리무버'
 '파우더워시' '클렌징티슈/패드' '패드' '시트팩' '워시오프팩' '패치' '필오프팩' '코팩' '슬리핑팩' '모델링팩'
 '선크림' '선쿠션' '애프터선' '선스틱' '선스프레이' '쿠션' '립스틱' '립틴트' '블러셔' '아이섀도우' '쉐이딩'
 '파운데이션' 'BB/CC' '아이라이너' '하이라이터' '마스카라' '아이브로우' '파우더/팩트' '컨실러' '프라이머/베이스'
 '립글로스' '립밤']


In [12]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = oliveyoung_df['중분류'] == '미스트/오일'
print(oliveyoung_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = oliveyoung_df['중분류'] == '스킨케어세트'
print(oliveyoung_df.loc[cond2, '대분류'].unique()) # 스킨케어

cond2 = oliveyoung_df['중분류'] == '패드'
print(oliveyoung_df.loc[cond2, '대분류'].unique()) # 마스크팩

['스킨케어']
['스킨케어']
['마스크팩']


In [13]:
# 패드 상품명 구체적으로 확인 : 토너 패드로 간주하고 대분류 자체를 스킨케어로 변경

cond = oliveyoung_df['중분류'] == '패드'
oliveyoung_df.loc[cond, ['대분류', '제품명']]

,대분류,제품명
200,마스크팩,더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획
201,마스크팩,더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획
202,마스크팩,더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트
203,마스크팩,메디힐 네모패드 100+100매 더블기획 7종 골라담기
206,마스크팩,메디힐 네모패드 100매 피부 고민별 골라담기
208,마스크팩,스킨푸드 캐롯 카로틴 카밍 워터 패드 60매+30매 한정기획 (+10매+캐롯시트 1매)
211,마스크팩,한율 5분 잡티톤업패드/트러블진정패드/모공청소패드 (달빛유자/어린쑥/부들밤)
212,마스크팩,메디큐브 제로모공패드 2.0 140매 기획 (70매 + 70매 리필)
214,마스크팩,넘버즈인 토너패드 리필기획 3종 중 택 1 (1번/4번/5번)
222,마스크팩,스킨푸드 캐롯 카로틴 카밍 워터 패드 기획(60매+30매 증정)


In [14]:
# 올인원 중분류 카테고리 확인 
cond = oliveyoung_df['소분류'] == '올인원' 
oliveyoung_df.loc[cond, '중분류'].unique() # 로션 or 기타 -> 세분카테고리 추가 의논 필요

array(['로션'], dtype=object)

In [15]:
# 통합 필요한 세부 카테고리 확인
cond1 = oliveyoung_df['대분류'] == '스킨케어'
print(oliveyoung_df.loc[cond1, '중분류'].unique())

cond2 = oliveyoung_df['중분류'] == '베이스메이크업'
print(oliveyoung_df.loc[cond2, '소분류'].unique())

cond3 = oliveyoung_df['중분류'] == '아이메이크업'
print(oliveyoung_df.loc[cond3, '소분류'].unique())

cond4 = oliveyoung_df['대분류'] == '마스크팩'
print(oliveyoung_df.loc[cond4, '중분류'].unique())

['크림' '에센스/세럼/앰플' '로션' '미스트/오일' '스킨/토너' '스킨케어세트']
['쿠션' '블러셔' '쉐이딩' '파운데이션' 'BB/CC' '하이라이터' '파우더/팩트' '컨실러' '프라이머/베이스']
['아이섀도우' '아이라이너' '마스카라' '아이브로우']
['패드' '시트팩' '페이셜팩' '패치' '코팩']


In [16]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['스킨', '토너']
skin = '|'.join(skin_list)
cond_skin = oliveyoung_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = oliveyoung_df['소분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '픽서']
fixer = '|'.join(fixer_list)
cond_fixer = oliveyoung_df['소분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = oliveyoung_df['소분류'].str.contains(shading)

primer_list = ['프라이머', '베이스', '톤업크림'] # 프라이머/베이스 컬럼에 (소분류)톤업크림 추가
primer = '|'.join(primer_list)
cond_primer = oliveyoung_df['소분류'].str.contains(primer)

for i in range(len(oliveyoung_df)):
  if cond_skin[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if oliveyoung_df['중분류'][i] == '패드': # 패드를 검색 단어에 포함시키면 '(소분류)클렌징/패드'도 스킨케어에 포함될 수 있음
      oliveyoung_df.loc[i, '중분류2'] = '토너/토너패드/스킨' 
    
    elif oliveyoung_df['중분류'][i] == '로션':
      oliveyoung_df.loc[i, '중분류2'] = '로션'

    elif oliveyoung_df['중분류'][i] == '에센스/세럼/앰플':
      oliveyoung_df.loc[i, '중분류2'] = '에센스/세럼/앰플'
    
    elif oliveyoung_df['중분류'][i] == '크림':
      oliveyoung_df.loc[i, '중분류2'] = '크림'

    elif oliveyoung_df['대분류'][i] == '마스크팩':
      oliveyoung_df.loc[i, '중분류2'] = '마스크팩'

    elif oliveyoung_df['대분류'][i] == '클렌징':
      oliveyoung_df.loc[i, '중분류2'] = '클렌징'

    elif oliveyoung_df['대분류'][i] == '선케어':
      oliveyoung_df.loc[i, '중분류2'] = '선케어'

    elif oliveyoung_df['중분류'][i] == '립메이크업':
      oliveyoung_df.loc[i, '중분류2'] = '립메이크업'
    
    elif oliveyoung_df['소분류'][i] == '컨실러':
      oliveyoung_df.loc[i, '중분류2'] = '컨실러'
      
    elif oliveyoung_df['소분류'][i] == '아이라이너':
      oliveyoung_df.loc[i, '중분류2'] = '아이라이너'

    elif oliveyoung_df['소분류'][i] == '아이섀도우':
      oliveyoung_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif oliveyoung_df['소분류'][i] == '아이브로우':
      oliveyoung_df.loc[i, '중분류2'] = '아이브로우'

    elif oliveyoung_df['소분류'][i] == '마스카라':
      oliveyoung_df.loc[i, '중분류2'] = '마스카라'
    
    elif oliveyoung_df['소분류'][i] == '속눈썹영양제':
      oliveyoung_df.loc[i, '중분류2'] = '속눈썹영양제'
      
    else: # 기획세트
      oliveyoung_df.loc[i, '중분류2'] = '기타'

In [17]:
print(oliveyoung_df['중분류2'].unique())
print(oliveyoung_df['중분류2'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['크림' '에센스/세럼/앰플' '로션' '파우더/팩트/픽서' '토너/토너패드/스킨' '기타' '클렌징' '마스크팩' '선케어'
 '파운데이션/쿠션/BB/CC' '립메이크업' '블러셔/쉐이딩/하이라이터' '아이섀도우' '아이라이너' '마스카라' '아이브로우'
 '컨실러' '프라이머/베이스']
18


,대분류,중분류,소분류,중분류2
0,스킨케어,크림,크림,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
2,스킨케어,로션,로션,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
...,...,...,...,...
495,메이크업,립메이크업,립틴트,립메이크업
496,메이크업,아이메이크업,아이브로우,아이브로우
497,메이크업,아이메이크업,마스카라,마스카라
498,메이크업,립메이크업,립틴트,립메이크업


In [18]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = oliveyoung_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = oliveyoung_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = oliveyoung_df['중분류2'].str.contains(eye)

for i in range(len(oliveyoung_df)):
  if cond_skincare[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if oliveyoung_df['중분류2'][i] == '클렌징':
      oliveyoung_df.loc[i, '중분류1'] = '클렌징'

    elif oliveyoung_df['중분류2'][i] == '마스크팩':
      oliveyoung_df.loc[i, '중분류1'] = '마스크팩'
      
    elif oliveyoung_df['중분류2'][i] == '기타':
      oliveyoung_df.loc[i, '중분류1'] = '기타'
      
    elif oliveyoung_df['중분류2'][i] == '선케어':
      oliveyoung_df.loc[i, '중분류1'] = '선케어'
    
    elif oliveyoung_df['중분류2'][i] == '립메이크업':
      oliveyoung_df.loc[i, '중분류1'] = '립메이크업'

In [19]:
print(oliveyoung_df['중분류1'].unique())
print(oliveyoung_df['중분류1'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '기타' '클렌징' '마스크팩' '선케어' '립메이크업' '아이메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,크림,크림,스킨케어,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
2,스킨케어,로션,로션,스킨케어,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
...,...,...,...,...,...
495,메이크업,립메이크업,립틴트,립메이크업,립메이크업
496,메이크업,아이메이크업,아이브로우,아이메이크업,아이브로우
497,메이크업,아이메이크업,마스카라,아이메이크업,마스카라
498,메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [20]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = oliveyoung_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = oliveyoung_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = oliveyoung_df['중분류1'].str.contains(colorclass)

for i in range(len(oliveyoung_df)):
  if cond_skinclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '색조메이크업'

In [21]:
# 대분류 고유값 확인 
print(oliveyoung_df['대분류'].unique())
print(oliveyoung_df['대분류'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,크림,크림,스킨케어,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
2,스킨케어,로션,로션,스킨케어,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
...,...,...,...,...,...
495,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업
496,색조메이크업,아이메이크업,아이브로우,아이메이크업,아이브로우
497,색조메이크업,아이메이크업,마스카라,아이메이크업,마스카라
498,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [22]:
# '(중분류)패드'의 분류값 확인
cond = oliveyoung_df['중분류'] == '패드'
oliveyoung_df.loc[cond, ['대분류', '중분류1', '중분류2']]

,대분류,중분류1,중분류2
200,스킨케어,스킨케어,토너/토너패드/스킨
201,스킨케어,스킨케어,토너/토너패드/스킨
202,스킨케어,스킨케어,토너/토너패드/스킨
203,스킨케어,스킨케어,토너/토너패드/스킨
206,스킨케어,스킨케어,토너/토너패드/스킨
208,스킨케어,스킨케어,토너/토너패드/스킨
211,스킨케어,스킨케어,토너/토너패드/스킨
212,스킨케어,스킨케어,토너/토너패드/스킨
214,스킨케어,스킨케어,토너/토너패드/스킨
222,스킨케어,스킨케어,토너/토너패드/스킨


## 립메이크업 중분류 세분화

In [23]:
# musinsa_df
oliveyoung_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,할인가,중분류1,중분류2
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,크림,크림,4.9,25321,59400,44500.0,스킨케어,크림
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000,24700.0,스킨케어,에센스/세럼/앰플
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,로션,로션,4.9,2688,53400,39900.0,스킨케어,로션
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500,27300.0,스킨케어,에센스/세럼/앰플
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000,32900.0,스킨케어,에센스/세럼/앰플


In [24]:
# 중분류2 복제
oliveyoung_df['중분류3'] = oliveyoung_df['중분류2']
oliveyoung_df['중분류3'].head()

0           크림
1    에센스/세럼/앰플
2           로션
3    에센스/세럼/앰플
4    에센스/세럼/앰플
Name: 중분류3, dtype: object

In [25]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = oliveyoung_df['중분류2'] == '립메이크업'
oliveyoung_df.loc[cond, '소분류'].unique()

array(['립스틱', '립틴트', '립글로스', '립밤'], dtype=object)

In [26]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

for i in range(len(oliveyoung_df)):
  if oliveyoung_df['소분류'][i] == '립글로스':
    oliveyoung_df.loc[i, '중분류3'] = '립글로스'

  elif oliveyoung_df['소분류'][i] == '립밤':
    oliveyoung_df.loc[i, '중분류3'] = '립밤'
    
  elif oliveyoung_df['소분류'][i] == '립스틱':
    oliveyoung_df.loc[i, '중분류3'] = '립스틱'
    
  elif oliveyoung_df['소분류'][i] == '립라이너':
    oliveyoung_df.loc[i, '중분류3'] = '립라이너'

  elif oliveyoung_df['소분류'][i] == '립틴트':
    oliveyoung_df.loc[i, '중분류3'] = '립틴트'


# 중분류3 입력값 확인
cond = oliveyoung_df['중분류2'] == '립메이크업'
print(oliveyoung_df.loc[cond, '중분류3'].unique())
print(oliveyoung_df.loc[cond, '중분류3'].nunique())
oliveyoung_df.loc[cond, ['소분류', '중분류3']].head()

['립스틱' '립틴트' '립글로스' '립밤']
4


,소분류,중분류3
401,립스틱,립스틱
402,립틴트,립틴트
404,립틴트,립틴트
408,립틴트,립틴트
409,립틴트,립틴트


## 데이터셋 정리

In [27]:
# 기존 카테고리 지우기
oliveyoung_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
oliveyoung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   평점      500 non-null    float64
 6   후기 수    500 non-null    int64  
 7   가격      500 non-null    int64  
 8   할인가     432 non-null    float64
 9   중분류1    500 non-null    object 
 10  중분류2    500 non-null    object 
 11  중분류3    500 non-null    object 
dtypes: float64(2), int64(3), object(7)
memory usage: 47.0+ KB


In [28]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = oliveyoung_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = oliveyoung_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
oliveyoung_df = oliveyoung_df[reorder]

oliveyoung_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', '할인가'],
      dtype='object')

In [29]:
# 최종 데이터셋 확인
print(oliveyoung_df.info())
oliveyoung_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류1    500 non-null    object 
 6   중분류2    500 non-null    object 
 7   중분류3    500 non-null    object 
 8   평점      500 non-null    float64
 9   후기 수    500 non-null    int64  
 10  가격      500 non-null    int64  
 11  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(7)
memory usage: 47.0+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,할인가
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400,44500.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000,24700.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400,39900.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500,27300.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000,32900.0


In [30]:
# 최종 데이터셋 저장
# oliveyoung_df.to_csv('oliveyoung_preprocessing.csv', index=False, encoding="utf-8-sig")